<a href="https://colab.research.google.com/github/tims457/ml_notebooks/blob/master/nlp/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>